In [1]:
import numpy as np
import pandas as pd

In [ ]:
# See database_helpers.py
run database_helpers.py

In [2]:
# Processing the gear is going to be slightly special. The source data
# was available in an ugly markdown table. "gear" values are a category,
# and "description" is really an aggregate of all the possible kinds of
# gear that fit that category. So we'll have to expand the right column to
# individual values and create a new "long" dataset
gear_descriptions = pd.read_csv("../Data/gear.csv")
gear_descriptions

,gear,description
0,bottom trawl,"bottom trawl, beam trawl, otter trawl, shrimp ..."
1,gillnets,"gillnet, trammel nets"
2,longline,"longline, pole and line, hand lines, lines"
3,other,"dragged gear, dredge, hand or tools, pots or t..."
4,pelagic trawl,pelagic trawl
5,purse seine,"purse seine, encircling nets"
6,small scale,"artisanal fishing gear, recreational fishing g..."
7,unknown,"unknown class, unknown by source, unknown by a..."


In [ ]:
gear_expanded = []

# Create a row that will be for any unmatchable values in this dimension
gear_expanded.append({"type": "Unknown", "name": "Unknown"})

for gear_description in gear_descriptions.to_dict("records"):
    gear_expanded.extend([{"type": gear_description["gear"].title(), "name": gear.strip().title()} for gear in gear_description["description"].split(",")])

# Create a dataFrame
dim_gear = pd.DataFrame(gear_expanded)

# Create an ID column values, using index
dim_gear.reset_index(inplace=True)
dim_gear.rename(columns={"index":"id"}, inplace=True)

# now insert into the database
gearInsertQuery = """
INSERT INTO "Dim_Gear"
("Id", "GearType", "GearName")
VALUES
%s
"""
bulk_insert(gearInsertQuery, dim_gear)